In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.autograd import Variable
from torch.utils.data import Dataset

import numpy as np
import random
import pandas as pd

# Visualization tools
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
import wandb
wandb.login()

wandb: Currently logged in as: cs22z105 (cs6910_assgnmt). Use `wandb login --relogin` to force relogin


True

In [3]:
sweep_config = {
    'method':'random'    
}

In [4]:
metric = {
    'name':'loss',
    'goal':'minimize'
}
sweep_config['metric'] = metric

In [5]:
parameters_dict = {
    'cell_type' :{
        'values':['RNN','LSTM','GRU']
    },
    'hidden_layer_size':{
        'values':[16,32,64,128]
    },
    'lr':{
        'values':[0.001,0.01,0.1]
    },
    'n_batches':{
        'values':[10,100]
    },
    'batch_size':{
        'values':[64,128]
    }
}
sweep_config['parameters'] = parameters_dict

In [6]:
sweep_id = wandb.sweep(sweep_config, project = 'Assignment_3')

Create sweep with ID: hgi601v9
Sweep URL: https://wandb.ai/cs6910_assgnmt/Assignment_3/sweeps/hgi601v9


In [7]:
SEED = 33
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [8]:
PATH_TO_DATA = "./aksharantar_sampled/hin/"

### Data Preprocessing

In [9]:
# storing all the alphabets of English and the pad char to a dictionary to create OHE representation later.
eng_alphabets = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
pad_char = '-PAD-'

eng_alpha2index = {pad_char: 0}
for index, alpha in enumerate(eng_alphabets):
    eng_alpha2index[alpha] = index+1

print(eng_alpha2index) #0B80 - 0BFF = 2944 - 3071

{'-PAD-': 0, 'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'H': 8, 'I': 9, 'J': 10, 'K': 11, 'L': 12, 'M': 13, 'N': 14, 'O': 15, 'P': 16, 'Q': 17, 'R': 18, 'S': 19, 'T': 20, 'U': 21, 'V': 22, 'W': 23, 'X': 24, 'Y': 25, 'Z': 26}


In [10]:
# Hindi Unicode Hex Range is 2304:2432. Source: https://en.wikipedia.org/wiki/Devanagari_(Unicode_block)

hindi_alphabets = [chr(alpha) for alpha in range(2304, 2432)]
hindi_alphabet_size = len(hindi_alphabets)

hindi_alpha2index = {pad_char: 0}
for index, alpha in enumerate(hindi_alphabets):
    hindi_alpha2index[alpha] = index+1

print(hindi_alpha2index)

{'-PAD-': 0, 'ऀ': 1, 'ँ': 2, 'ं': 3, 'ः': 4, 'ऄ': 5, 'अ': 6, 'आ': 7, 'इ': 8, 'ई': 9, 'उ': 10, 'ऊ': 11, 'ऋ': 12, 'ऌ': 13, 'ऍ': 14, 'ऎ': 15, 'ए': 16, 'ऐ': 17, 'ऑ': 18, 'ऒ': 19, 'ओ': 20, 'औ': 21, 'क': 22, 'ख': 23, 'ग': 24, 'घ': 25, 'ङ': 26, 'च': 27, 'छ': 28, 'ज': 29, 'झ': 30, 'ञ': 31, 'ट': 32, 'ठ': 33, 'ड': 34, 'ढ': 35, 'ण': 36, 'त': 37, 'थ': 38, 'द': 39, 'ध': 40, 'न': 41, 'ऩ': 42, 'प': 43, 'फ': 44, 'ब': 45, 'भ': 46, 'म': 47, 'य': 48, 'र': 49, 'ऱ': 50, 'ल': 51, 'ळ': 52, 'ऴ': 53, 'व': 54, 'श': 55, 'ष': 56, 'स': 57, 'ह': 58, 'ऺ': 59, 'ऻ': 60, '़': 61, 'ऽ': 62, 'ा': 63, 'ि': 64, 'ी': 65, 'ु': 66, 'ू': 67, 'ृ': 68, 'ॄ': 69, 'ॅ': 70, 'ॆ': 71, 'े': 72, 'ै': 73, 'ॉ': 74, 'ॊ': 75, 'ो': 76, 'ौ': 77, '्': 78, 'ॎ': 79, 'ॏ': 80, 'ॐ': 81, '॑': 82, '॒': 83, '॓': 84, '॔': 85, 'ॕ': 86, 'ॖ': 87, 'ॗ': 88, 'क़': 89, 'ख़': 90, 'ग़': 91, 'ज़': 92, 'ड़': 93, 'ढ़': 94, 'फ़': 95, 'य़': 96, 'ॠ': 97, 'ॡ': 98, 'ॢ': 99, 'ॣ': 100, '।': 101, '॥': 102, '०': 103, '१': 104, '२': 105, '३': 106, '४': 107, '५': 108, '६': 109, '७': 

In [11]:
# Funcitons used to do some pre-processing.
# removing all non-alphabetic char in English as well as Hindi.

import re
non_eng_letters_regex = re.compile('[^a-zA-Z ]')

# Remove all English non-letters
def cleanEnglishVocab(line):
    line = line.replace('-', ' ').replace(',', ' ').upper()
    line = non_eng_letters_regex.sub('', line)
    return line.split()

# Remove all Hindi non-letters
def cleanHindiVocab(line):
    line = line.replace('-', ' ').replace(',', ' ')
    cleaned_line = ''
    for char in line:
        if char in hindi_alpha2index or char == ' ':
            cleaned_line += char
    return cleaned_line.split()

In [12]:
class TransliterationDataLoader(Dataset):
    def __init__(self, filename):
        self.eng_words, self.hindi_words = self.readDataset(filename, cleanHindiVocab)
        self.shuffle_indices = list(range(len(self.eng_words)))
        random.shuffle(self.shuffle_indices)
        self.shuffle_start_index = 0
        
    def __len__(self):
        return len(self.eng_words)
    
    def __getitem__(self, idx):
        return self.eng_words[idx], self.hindi_words[idx]
    
    def readDataset(self, filename, lang_vocab_cleaner):
        '''Task : to read the csv file and store all the contents in a list.
                  Then we will do some pre-processing of data to remove noise as well as delimeters. '''
        transliterationCorpus = pd.read_csv(filename,header=None)
        lang1_words = []
        lang2_words = []

        for index,line in transliterationCorpus.iterrows():
            wordlist1 = cleanEnglishVocab(line[0]) # clean English words.
            wordlist2 = lang_vocab_cleaner(line[1])# clean hindi words.

            # Skip noisy data
            if len(wordlist1) != len(wordlist2):
                print('Skipping: ', line[0].text, ' - ', line[1].text)
                continue

            for word in wordlist1:
                lang1_words.append(word)
            for word in wordlist2:
                lang2_words.append(word)

        return lang1_words, lang2_words
    
    def get_random_sample(self):
        return self.__getitem__(np.random.randint(len(self.eng_words)))
    
    def get_batch_from_array(self, batch_size, array): # child function of get_batch() function.
        '''Given an array , and batch size , this fucntion will return some samples from the array i.e can be HindiWords or EnglishWords etc. '''
        end = self.shuffle_start_index + batch_size # what index till i want to go.
        batch = []
        if end >= len(self.eng_words): # if we overflow the words array , we have to loop back.
            batch = [array[i] for i in self.shuffle_indices[0:end%len(self.eng_words)]]
            end = len(self.eng_words)
        return batch + [array[i] for i in self.shuffle_indices[self.shuffle_start_index : end]]
    
    def get_batch(self, batch_size, postprocess = True):
        eng_batch = self.get_batch_from_array(batch_size, self.eng_words)
        hindi_batch = self.get_batch_from_array(batch_size, self.hindi_words)
        self.shuffle_start_index += batch_size + 1
        
        # Reshuffle if 1 epoch is complete
        if self.shuffle_start_index >= len(self.eng_words):
            random.shuffle(self.shuffle_indices)
            self.shuffle_start_index = 0
            
        return eng_batch, hindi_batch

In [13]:
train_data = TransliterationDataLoader(PATH_TO_DATA+'hin_train.csv')
val_data = TransliterationDataLoader(PATH_TO_DATA+'hin_valid.csv')
test_data = TransliterationDataLoader(PATH_TO_DATA+'hin_test.csv') 

In [14]:
#train and test data directory


train_data1 = pd.read_csv("./aksharantar_sampled/hin/hin_train.csv",header=None)
val_data1 =  pd.read_csv("./aksharantar_sampled/hin/hin_valid.csv",header=None)
test_data1 =  pd.read_csv("./aksharantar_sampled/hin/hin_test.csv",header=None)

train_data1.head()

,0,1
0,shastragaar,शस्त्रागार
1,bindhya,बिन्द्या
2,kirankant,किरणकांत
3,yagyopaveet,यज्ञोपवीत
4,ratania,रटानिया


In [15]:
for index,line in train_data1.iterrows():
    print(line[1])

शस्त्रागार
बिन्द्या
किरणकांत
यज्ञोपवीत
रटानिया
वागण्याचे
देशभरामध्ये
सुघड़पन
मोहीवाल
सर्वसंग्रह
बसेको
तुमच्यापैकी
कान्यकुब्ज
इनटॉक्सिनेशन
मेच्यूरिटी
अगरी
अनुक्रमानुपात
धूलचन्द
अवलेह
एबरोर्ड
बैलर्स
बार्गली
पंक्चर्ड
हैंकर्स
जवानोंके
पौंकी
जगनधाम
पोन्नियम
आईएनबीए
वेदलम
चिनवास
मारवाड़ा
अनसमझा
इस्टूडेंट
दुःखीत
सिघांची
शिक्षेचे
उज्वलतम
आपट्टन
अम्बिकावन
खगनी
थियामिन
सोशियोलाजिकल
एप
भाजपाको
इस्तिथि
छायाटांड
तोपवाल
गुदवाते
विसलेरी
थर्मलपावर
सांचे
पिंटिया
कैंटाबिल
अरेंजिंग
वाहिन्यांसाठी
आईटीपीए
पिक्सलमेटर
पहनायूंगा
मेडसन
विदयापीठ
रुपयेची
नजरबन्दी
क्रीचन
पेशनधारियों
देखलूँ
मैकलेड
लोकरचनाओं
गोविंदापुरा
आईएफएन
रॉटवीलर
ईश्वरपुर
चारित्र्य
एक्सप्लोशन
यास्मिनला
त्रिकाकरा
सतनाजा
पूछें
धनसंकट
काइपरनिक
तेजस्वीय
श्नाइडर
बन्हुक
बयानकिया
मधुमोम
सु
ग्लूज़
चिल्लाकर
जीआईटीएएम
श्रीकला
लैम्पों
अपायल
नेफियू
एंक्लेव
भुलानपुर
विकेंद्री
इंफेंट
लाठियाँ
ढाऊँगा
किमैट्रिक
गौतमाचे
फरोटका
आमय
चंदनहारी
डेकाथॉन
एग्रेट
पुरखवा
हाइपोग्लाइसीन
महवा
दूभ
अरजिन्द्र
शातिनिकेतन
अमरगुन
भावपुर
ट्रांसक्रिप्टेज़
गंतिविधियों
मरुकांतर
होमबल

अटेली
त्सार
बुदबुदाऊंगा
हैम्बर्ब
गोमार्ट
सहस्रभुज
अरूणदेव
एनआरयूसीसी
नेटको
कट्टरताएं
संपर्काची
अश्विस
बगरवारा
एक्सानमोबिल
तैयारियां
हार्डकोरटीन
दौड़ाक
दोलिन्सकी
तीतरों
दुर्घटनांत
आसमति
अपव्वय
लेसिका
एक्सामिनर
मुहावरो
गृहता
इन्टलेक्चुअल
बढ़ोई
इथिल
पार्थगली
क्रैंकपिन
थेक्कालूर
सृष्टिकर्ता
ह्नेनसांग
लागलपेट
फर्गल
फ़ायर्ड
बनसिंह
इंस्टुमेंट
वारूंगसे
कांट्रास्ट
सेनाभवन
कुवलयमाला
गोगोज
एक्टिनिडेन
ज़ुका
बढ़ले
एक्यूप्रेक
महाभारतको
जलगाओंकर
इमलियाचट्टी
रण्डीपन
लालास्राव
हुलवा
खुराफ़त
वेसेली
खुलेने
एससीसीएसएल
स्प्रेडट्रूम
लाडाचा
हनचुआं
बाजला
पक्षासमवेत
इगनीशन
अमाइताल
तुमरो
स्पायरवुड
संगीतकारों
काहुला
गोडाले
सुनिलाख्या
फनवा
यथो
घरसंवाददाता
रैम्बो
पासबुकों
तारोदेवी
दर्पणों
मारुथल
पीकेजी
बिटकिन्स
मोनसितलाह
रणनीतिकारी
गौचेंग
हॉटेलचालकांकडून
हृदयविदीर्ण
एमवाज़ी
आईएएसएस
कुतारिया
चहचहाती
रोटबेल
पंडोखर
वलनरेबल
झौंकने
कोपरेशानी
निमफेनबर्ग
करोआने
तिबरानी
वहिल्या
जिनेश
वाइज़र
मेरिनियाक
भगनवान
हिंदीभाषीय
टेलीपोर्टेशन
उत्तराखंडात
चफुआ
भरियाड़ी
तपोसिनी
विकिकोश
इच्छामत्यु
एक्सटिंक्शन
कठोरपरिश्रम
डींगे
वधिष्ट
मै

डिलिवर
सुग्गड़
चैबदार
राशनवाला
खोजीराम
महानगरांप्रमाणेच
वायलिनिस्ट
वाटमोर
विकरालतम
पटकती
तलस्पर्शी
वितराग
ठहरानेवाली
सामन्तवाद
बैचों
अंतर्संबंधों
कावासाजी
लैकलिस्टिंग
एल्थम
ट्रेवलिंग
पाइला
कोठेश्वरनाथ
विसूचिका
नंदगवा
लगवाड़ा
उठादी
रायग्राम
श्रीगणेशन
फुजाई
रतियों
मश्रिके
एक्स्प्रेसडिब्रूगढ़
फिलिप्सलैंड
तस्तोत्सी
एन्डरो
देनवाला
इन्वेस्टिगेटर्स
प्राणप्रतिष्ठित
बरदही
तनख्वाहें
आषाढ
केतौरपरहुई
फरासखाना
वहाबियों
वैक्युम
बीकाभाई
व्याजी
उमड़ेगा
मरजोरम
अक्सिरियत
मोखल
बिराजता
टोनेगावा
प्रभागचंद
चर्यजन
रियायतें
उधरित
अंकालम्मा
बोद्धिकी
माफीपत्र
राउंडवर्म
एक्सटै्रक्टर्स
खानेंफैक्टरियां
वेंकेट
परिपक्वन
मकाहा
हालगेरे
जयसेखर
मिश्रपिकांची
राजौरा
फेट्स
सालमोनेला
वेंदिन
सजीदुल
ब्लेझरची
इनडेमनिटी
संगमजी
गैसीफायर
एमएसएमईएस
तानेश्वर
शक्तिकरण
कुंभले
एन्टीविजन
कुसमाहा
सल
वन्नामेई
दैत्याकार
गजकरण
काटुंबा
रोसेशिया
नेसून
स्टेन्कोविच
सीमाप्रश्नाचा
गुलवतन
फुल्लरा
डेडगांव
करोड़ी
टेव्स
यूटीआईआईसएएल
गडग
काभोग
फैलाना
शंभू
जड़गलन
वस्त्राभाव
शिक्षकांच्यात
पुर्नेय्या
पर्यटनमंत्री
सतकोसिया
क्लोनी
सीतारामय्या
अलॉटमेंट्स
न

कालूलाल
यांजलि
कासिमकोटा
विधायकसंदीप
पठानमथिटा
वाटेवरच्या
ड्रोगो
प्रवृत्तियों
दुसरों
सिंधव
प्रेतावर
फ़ेथ
किस्मतिया
अठारामुरा
बेन्कनस्टीन
लिन्टनर
नेवियो
सकुलपुर
इपेडीमिओलॉजिकल
नीमला
आईपीएसएस
प्रीधि
ओडी
सरनेम
जगदीश्वरा
केजीबीवी
दाबड़े
रुकवाल
शास्त्रपूर्ण
ट्रिप्टेन
आरगुमेंटेटिव
कुसुम्भी
ट्रुथवेन
घटकामध्ये
पक्षातील
प्रत्येकाना
मक्केका
अखाड़ची
भमाशा
फर्नेलमोंट
मेंप्रस्तुत
करनकुमार
पुट्टापर्ती
षण्मुखरत्नम
प्रौढ़ावस्था
अहिलावती
इस्तेमालदिल्ली
सब्रवाल
कांगकोपई
टीयों
सासारामी
अपर्याप्तं
पुट्टापुर्थी
सूग
फेलेप्स
बटोरने
ऐरावतेश्वर
धनदंड
भापड़ी
क्षत्रभंग
स्ट्रैची
बदलायची
अरलैंडो
यानयुआन
वाजेवाडी
झाबख
खवाउत
हैस्टिंग
रेलसीसी
चारुवली
गोविंदजीवाला
शहरों
पंगुडवाले
लौटकर
चंदगांव
ज्ञानकोश
सिंगापुरवासियों
रहोब
इनवारोमेंट
अस्पस्ट
नैवेद्यम्
बांसबल्ली
अभयचंद्र
पाँचोटा
बिछवा
टोमोटो
बोफर्स
वांदेवेघे
ढटे
डेसर्टिफिकेशन
उपमहाद्धीप
ब्रजकला
पैप्लेट
उदेख
तिहास
सुतीर्थ
पद्महस्ता
दारिमी
पेड़ना
विमुखी
दिलवाड़ा
जायदाद
किसैनी
सरदौली
क़बालियों
स्क्रीचर
अरातियात्
क्रितेंद्र
लेटाना
प्रेस्बिटेरियन
शिवेंद्रराजेंकडून
बायोसेक्य

अव्यक्त
गण्डास्थि
हालचालीबाबत
धूएँ
मृत्युप्रकरणाची
फ़्यूरफ़्यूरल
कलागुणांचे
खौशलीम
जनपदों
परधानी
बैसापुर
संपंगी
फील्स
उष्माघाताचा
झांकते
फेरेली
लोरकन
पटांडे
अर्थविज्ञ
पिष्टि
सिमुलेशन
खरदूषण
अनारवाला
शशुद्धिकरण
तणुकु
चंदकिशोर
कलमगंज
ब्रिजरानी
अमीयापुर
होर्स्ट
रमणीयार्थप्रतिपादकः
घोषणाओं
मानपूर्वक
एंजियोप्लस्टी
पुनर्घोषणा
मेंचिदंबरम
गोदाने
जाडीकेचर
ककपोरा
धूलभरे
चिलहटा
वाटण्यासारखी
रुक्मिणीचे
चार्जेंज़
जिम्मेद
फॉरिन
स्त्रीप्रकृति
तरुणेश
लहवीत
निरूक्तकार
बढ़ावें
जीडीडी
नयूली
अरपा
कास्पर्सकी
किचौचा
क्यूजी
एनस्थेटिक
पोडलिप्निक
क्वारिंग
अनावृत्त
गुरुवारवीरवार
बीयरपार्टी
रायबरेली
पीड़ावश
भारतवैद्यक
स्टेनोकार्डिया
सुन्देश
हिरियुरू
प्रातिनिधि
बमेरिका
छपवाने
आज्ञानी
जनित्र
लगावें
पुक्कुसाति
नलवात्रू
डोग्मा
कैलिमेयर
केरना
मेत्तुपलायम
अटाकामा
अथवानी
इंपरफेक्टो
यौक्तिक
गजवाल
कर्मबीर
हेडमास्टरनी
शिनहुआ
उपस्थितिमें
पुरस्कारांचे
लूलिंक
पचमो
संधीला
एर्गोनोमिक
राजागंज
मात्सुशिमा
ईशकरण
कल्लारीपायाट्टू
कैंडलमार्च
अरुलाल
सुसंगत
तिरुर
महमुदल
खुपिया
परवरदिगारा
नयामत
हूच
फुलावर
ग्वारडिया
मंत्रालयमें
एन्यूमरेश

...

रियायत
पर्याप्तु
गुरवेज
परकृति
शिशोलाव
आदिंचा
स्वामीशरण
अंकुरदीप
कोचट्टा
सोफास
ओरेगैनो
दिखावापन
डाँटेंगी
फिनमेकनिका
छेदब्रिटिशभयंकर
सच्चा
छैवर
पैनग्रांगो
कीमनाई
महुवाखेडी
गतौरा
शौचालयों
मैक्डॉवेल्स
नवस्वाधीन
फ्रैंजिक
बेटीके
वगेरे
आईक्यूएसी
टेनेक्स
स्टोफाली
देमेत्री
रसाधिपति
अतिराजपुर
तिगरिस
मर्सिनरी
सिंघोही
अंडरव्लर्ड
जौरियां
कंफेशनरी
मणिकंठन
महाव्रतरूपी
मौनगुरु
धूमाधाम
खामख्वाह
वायगोत्स्की
मुरुंताउ
संभाषणाची
सद्यस्थितीचा
वावरिंका
भिगाने
मेदुवडे
पद्दोन्नति
शोइची
तमालिका
पाम्पेई
जानवरके
नखांची
हेल्त्सुक
फलूदों
चार्जमध्ये
पुलुमावी
सौहरे
रेस्पायरेट्री
तेजाजीनगर
योरेन्स
राजका
साकारल्यास
पशाकोटा
अवमानच
सम्मलनों
ननैया
हरामज़ादा
गांधीवादी
खेलनें
नयुक्तियों
अविलमब
नवाजिश
इतिहासकारों
बहिर्विवाही
पकेंगी
शोधपत्रों
योजनाच
राष्ठ्र
द्रमणि
यूसुफजई
गाजिउद्दीन
धर्मोपदेशनविधौ
हेंगोर
विनियोगों
एंबुलडेनिया
आपातकालीन
कोइरियापार
स्नानपूर्व
लेजरद्वारे
श्रीमतियों
पीडियेट्रिक्स
वलशाली
लूटेरों
हथखेलियां
अभिष्यन्द
अहवाज
गढीच्या
निद्रावस्था
दुगाना
छरोड़
अंतर्राष्ट्रीय
प्रदोश
ओग्
प्रतिवेदनो
केजीएमओए
चंद्रकांतने
सा

अनिष्ट
प्रेमनगरी
पोर्टू
टर्मिनस
ठीककरने
रोहिक
खटरेजा
मांसपेश
तिदांग
अनोखिन
अर्थसंकल्पाचे
गोशाईंगंज
सुधननिधि
असख्य
सिंहपुर
नरना
चिंताला
सुहागरस
कुलधुफ़ुशी
कलीसे
हठयोगिनी
मृतमश्नुते
वैहनी
जुनेज
लजीजता
कार्टेस
विद्याध्ययन
जेटीडीसी
पैथोलाजिस्ट
आईआरओएफ
उठवाल
पालवंश
कर्जिकन
दुआएं
लोहासन
हंसने
शिल्पास
सौकिलोमीटर
सागरों
मुकाबलाः
लीव्स
गाड़कार
केहेव
पैशांवर
महेन्द्रनाथ
आकार्षि
भूचा
मिंजी
शिलापर
कफाला
रणवीक
हैरानियां
गुरमेलो
खेल्लानी
दोरेजे
टेस्सा
स्वीकारणारे
बेवफूफ
पैरा
सौहार्दतेची
स्मार्टबॉल
छीकना
प्रावारीय
सीमावती
स्पासजेट
द्वंद्व
चढ़ाएगा
आम्बेडकरवादी
मायलम्मा
धनारिया
सेवनिवृत्ति
अब्दालीला
आसतीनों
उपनद्यांच्या
हुनैन
बचनी
सामानाचे
स्पोंडोलाईटिस
शेल्मर
संक्रान्त
मुज़म्मल
डिविलरी
सरण्यु
केडीएस
कोवालचुक
नंदप्रकाश
मोमोना
अनुभूयते
सूजीबहार
कोलाइबेली
देज्यांग
बहकना
खुरपैंच
बढ़लैन
न्यूट्रीशिनिस्ट
निरामिषभोजी
होथियार
दलहाने
शिवाचार्यों
लेगिट
राजस्वा
सैनेटाइजेशन
पिनवाड़ा
शिवपाड़ा
राजकोविक
पुस्बा
केशवन
पोजीशन्स
बेनटेन
भुनाके
प्राइमड
मात्रापतन
सुसिंद्रो
गुरुओं
गाबली
अकलाख
दीनूबा
सिम्बाल्टा
नाटकमें
डिविली

दयानन
ग्रन्थसूची
स्पेंशल
माइक्रोसॉप्ट
देखायेगा
नरहरपुर
घुमचे
कौनसरनाग
अयासिकुर
बेबीफेस
संपादनो
जेफस
बृहस्पिति
घपलासन
जेलास्को
जलसंधि
संस्थानांचे
बिछौने
चक्रामपुर
द्विपेन
संभालके
मधाराम
कैरीफोरम
धजावला
क्षेत्राचे
ऑस्ट्रेलियामे
अनुकूलनों
अपरम्पार
उदवाडा
लिजिंग
डिग्गाक
स्पर्मेटोजीनस
पेडेंट्स
लैतोंजाम
जुन्नोर
बरोल्ली
गाइटलाइंस
मूर्तिभञ्जक
अस्कोटी
त्रोद्योग
आदेशाद्वारे
अभ्यासी
शिरोल
शागिल
सेन्दाई
पतिघर
समझोत्याचे
वाईंडिंग
छिकहरा
नापोगे
दिशागत
शपथपत्र
तुलसीघाट
वर्षपति
सिन्जेन्टा
कजोड़लाल
झांगझू
डिकाडला
बहोतों
ल्यूटिन
मिरतून
मातृभाषेने
संस्कृतीप्रमाणे
पांचलवाड़ा
सुक्षदेव
लेनदारियां
घृतवंशी
पापहरण
क्लॉर्क
वृत्तचित्रकार
इबेरिक
मीनलदेवी
एलर्जिस्ट
निदेंशों
ग्रोनो
वाउथरिंग
इनफेक्टिड
फलैसा
सिंधुसागर
पडनेवाला
दांड्याची
नौकरसाह
बीपीसीएल
प्रयागनारायण
लेईमाखोंगा
पीएस
अबॉलिशन
सिहोड़ी
सीआईएमए
गम्भीरवस्था
श्यामली
लायमाउथ
लकीशों
ध्याते
एन्जर्स
लंजता
खभोग
कीस्वेटर
विरोधकांसह
सीखचे
ओटले
उनकाध्यान
खिलांरा
खटकट
जनायाब
बारामूडा
दालमण्डी
गंवाई
लॉरेंजो
बौधाश्रम
मेंटेंनेंस
विद्रोहीजी
इशिला
साइट्रोन
लिटिगेटन
प्रगति

झमरेड़ा
चौपदरीकरण
पगड़ीबंध
जमताड़ा
द्व्यक्षर
बौकाला
हाइपोथायरॉइडिस्म
गुड्डियो
घघोटिया
फ़ाइलिंग
भुमिकाओं
दागिन्याचा
घाबरू
दुर्गेची
अंशुला
मछलियाँ
हवाइयां
बप्पाचे
पिकासावेब
पासके
बुटेरी
शोलामारी
मोसल्लम
यूजीएमएसी
पचवी
अंचभित्त
बेडौलता
जिनां
गैमर
जीवाष्म
ड्रेंसिग
बार्षिक
गुणनखण्डों
पछाइत
अलारसा
एशेलॉन
सुब्रतरॉय
मुल्तान
मार्शलों
कुदानी
पाराहेड़ा
श्रुणानंद
अधिवेशनामध्ये
मिनकानाडी
सीटैक
लखमसी
मिसुर
कविकोकिल
सलैब
सातपैकी
जीवाणूंचे
उर्रा
चांदपा
पौडेल
छावण्यादेखील
बख्तावर
जोनाकर
घोलेंगी
चिकनिया
घुमल
डैड्रफ
तितरवाड़ा
उथेनकला
हंॅसते
उभयलिंगाश्रयी
मैलापाण्याचा
वालीबाल
बकेसू
स्वप्नाकडे
ब्लीनी
सबकेचलते
लालफीताशाही
किशने
यांगयान
गुंडिले
मेज़ू
धुरैरी
चंद्रेल
जीहें
टेंडरमध्येच
आखिरक्यों
सहम
रिचमंड
सिंडिया
सभिक
छमके
पिचेश्वर
डिरेक्शंस
निस्सारित
अहतेशाम
पारिस्थितिकीतंत्र
मट्टू
विषमलिंगी
बंजारबारू
अभिव्यक्तिके
व्रतधारण
छापूँ
नाज़िरा
डेस्टीमनी
चिट्ठोंकी
तोतसंगला
थ्रश
परछाइयों
मुरसालीन
सनाहली
साहित्यभूषण
कुलियों
ग्रूनी
चच्चे
गुरूवे
मुइंच
आत्मस्पर्शी
फिरात
मालमोटारीवर
चिलारीच्या
बाउंडीवॉल
स्वर्णजयंती
वांझोउ


उसिप्रकार
बैमबर्डिगे
सोडल्यागत
ग्लोबलिस्ट
क्रस्टेसियन
नलपुर
बिखरे
एनपीएमपी
मनाज़िर
बद्रीपुरा
आसौदा
मकमूल
तिकारी
टैमी
करेंटर
नैनोस्कोपी
दगती
पोर्न्स्टार
नर्तकी
गुप्तांगों
न्योंगब्योन
गुजरों
पाथर्डीसारख्या
सिहानीगेट
प्रसन्नमुखी
मैंडोलिन
रंतिदेव
डाँवाडोल
कागजबोर्ड
ग्रेना
सत्ताधिकार
तानेज
यूएनओएम
कूश
सगौर
शिखंडिनी
पोस्टमध्ये
सर्वलक्षणयुक्तता
टुक्लोर
वीरपाड़ा
ट्रैजिक
किनरुई
सदरातील
बैकबोन
सुखसोयी
कार्यकारिणीमे
अकांश
उच्चवंश
केपेल
विधादेवी
कोबाल
सामग्रीअधिक
उजिनोल
समंदरों
हरीपुरा
विदेता
कागदपत्रांवरून
जुहारा
हूतिया
पद्भनाभपुर
बुतरस
ग्रेटअप
हुगेस
छिनीं
सेवकावर
लथाड़
कलकटिया
खरेया
मालेगावी
मौलेन्दु
देसालियन
कर्मदाता
कितसुकी
हेनी
मंदिना
बरूला
एलएलजी
शार्प
दलकिर
लेयर्ड
खटखटात
रामदियाल
अपक्षेओं
अकादमीची
सिगनेटरी
रोस्लर
कर्तज्ञ
तेजवानी
नूरद
शूर्पनखा
इन्वोक
अनंतमणि
सुकविंदर
खोइर
तुगलकनाम
मूलिया
खुखनियाड़ा
देवजरा
निकनलने
निष्पीडन
अंधाधुंन
अलहलबी
सीसीपीएस
इंडोसन
काव्यामृत
प्राणवंत
मोदवाडिया
भैरगोंडा
अचीब
सैमसेंग
एट्मोस्फीयर
येरागी
नीलांबुजा
खींचो
साल्वेंट
डिसिप्लिन्स
लेक्टोफेरिन
उपकक्षाओं
फॉस्फोरस
बज

In [17]:
print(f"Number of training examples: {len(train_data1)}")
print(f"Number of validation examples: {len(val_data1)}")
print(f"Number of test examples: {len(test_data1)}")

Number of training examples: 51200
Number of validation examples: 4096
Number of test examples: 4096


In [18]:
train_data.get_batch(10)

(['WUHANMADHUN',
  'BIGADENGE',
  'CHAKRAAYE',
  'REDSEAR',
  'BHAVABHUSHAN',
  'RAAMMANOHAR',
  'THAYOL',
  'BANGDA',
  'LAKKHO',
  'SIGANPORIA'],
 ['वुहानमधून',
  'बिगाडेंगे',
  'चकराए',
  'रेडसीअर',
  'भवभूषण',
  'राममनोहर',
  'थायोल',
  'बांगदा',
  'लक्खो',
  'सिगनपुरिया'])

In [19]:
## Encoding the words

def word_rep(word, letter2index, device = 'cpu'):
    rep = torch.zeros(len(word)+1, 1, len(letter2index)).to(device)
    for letter_index, letter in enumerate(word):
        pos = letter2index[letter]
        rep[letter_index][0][pos] = 1
    pad_pos = letter2index[pad_char]
    rep[letter_index+1][0][pad_pos] = 1
    return rep

def gt_rep(word, letter2index, device = 'cpu'):
    gt_rep = torch.zeros([len(word)+1, 1], dtype=torch.long).to(device)
    for letter_index, letter in enumerate(word):
        pos = letter2index[letter]
        gt_rep[letter_index][0] = pos
    gt_rep[letter_index+1][0] = letter2index[pad_char]
    return gt_rep

In [20]:
eng, hindi = train_data.get_random_sample()
eng_rep = word_rep(eng, eng_alpha2index)
print(eng, eng_rep)

PADHAARANE tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [21]:
hindi_gt = gt_rep(hindi, hindi_alpha2index)
print(hindi, hindi_gt.shape[0])

पधारने 7


## Encoder-Decoder

In [22]:
MAX_OUTPUT_CHARS = 30
class Transliteration_EncoderDecoder(nn.Module):
    
    def __init__(self, input_size, hidden_size, output_size, verbose=False):
        super(Transliteration_EncoderDecoder, self).__init__()
        
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.encoder_rnn_cell = nn.GRU(input_size, hidden_size)
        self.decoder_rnn_cell = nn.GRU(output_size, hidden_size)
        
        self.h2o = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=2)
        
        self.verbose = verbose
        
    def forward(self, input, max_output_chars = MAX_OUTPUT_CHARS, device = 'cpu', ground_truth = None):
        
        # encoder
        out, hidden = self.encoder_rnn_cell(input)
        
        if self.verbose:
            print('Encoder input', input.shape)
            print('Encoder output', out.shape)
            print('Encoder hidden', hidden.shape)
        
        # decoder
        decoder_state = hidden
        decoder_input = torch.zeros(1, 1, self.output_size).to(device) # also can be variable i.e learned from data.
        outputs = []
        
        if self.verbose:
            print('Decoder state', decoder_state.shape)
            print('Decoder input', decoder_input.shape)
        
        for i in range(max_output_chars):
            
            out, decoder_state = self.decoder_rnn_cell(decoder_input, decoder_state)
            
            if self.verbose:
                print('Decoder intermediate output', out.shape)
            
            out = self.h2o(decoder_state)
            out = self.softmax(out)
            outputs.append(out.view(1, -1))

           
            if self.verbose:
                print('Decoder output', out.shape)
                self.verbose = False
           
            max_idx = torch.argmax(out, 2, keepdim=True)
            
            if not ground_truth is None:
                max_idx = ground_truth[i].reshape(1, 1, 1)
                
            one_hot = torch.FloatTensor(out.shape).to(device)
            one_hot.zero_() # all the elements will be 0.
            
            one_hot.scatter_(2, max_idx, 1)
            
            decoder_input = one_hot.detach() # don't pass gradient with this tensor.
            
        return outputs

In [23]:
# inference routine for sequence model.
def infer(net, eng_word,shape,device ='cpu'):
    input_ = word_rep(eng_word,eng_alpha2index,device) # convert the name into one hot encoding.
    outputs = net(input_,shape,device) # initilise the hidden layer.
    
    return outputs

In [24]:
net = Transliteration_EncoderDecoder(len(eng_alpha2index), 256, len(hindi_alpha2index), verbose=True)

In [25]:
out = infer(net, 'INDIA', 30)

Encoder input torch.Size([6, 1, 27])
Encoder output torch.Size([6, 1, 256])
Encoder hidden torch.Size([1, 1, 256])
Decoder state torch.Size([1, 1, 256])
Decoder input torch.Size([1, 1, 129])
Decoder intermediate output torch.Size([1, 1, 256])
Decoder output torch.Size([1, 1, 129])


In [26]:
print(len(out))
for i in range(len(out)):
    print(out[i].shape, list(hindi_alpha2index.keys())[list(hindi_alpha2index.values()).index(torch.argmax(out[i]))])

30
torch.Size([1, 129]) ऊ
torch.Size([1, 129]) ऊ
torch.Size([1, 129]) ऍ
torch.Size([1, 129]) ऍ
torch.Size([1, 129]) ऍ
torch.Size([1, 129]) ऍ
torch.Size([1, 129]) ऍ
torch.Size([1, 129]) ऍ
torch.Size([1, 129]) ऍ
torch.Size([1, 129]) ऍ
torch.Size([1, 129]) ऍ
torch.Size([1, 129]) ऍ
torch.Size([1, 129]) ऍ
torch.Size([1, 129]) ऍ
torch.Size([1, 129]) ऍ
torch.Size([1, 129]) ऍ
torch.Size([1, 129]) ऍ
torch.Size([1, 129]) ऍ
torch.Size([1, 129]) ऍ
torch.Size([1, 129]) ऍ
torch.Size([1, 129]) ऍ
torch.Size([1, 129]) ऍ
torch.Size([1, 129]) ऍ
torch.Size([1, 129]) ऍ
torch.Size([1, 129]) ऍ
torch.Size([1, 129]) ऍ
torch.Size([1, 129]) ऍ
torch.Size([1, 129]) ऍ
torch.Size([1, 129]) ऍ
torch.Size([1, 129]) ऍ


## Encoder-Decoder with Attention

In [27]:
class Transliteration_EncoderDecoder_Attention(nn.Module):
    
    def __init__(self, input_size, hidden_size, output_size, verbose=False):
        super(Transliteration_EncoderDecoder_Attention, self).__init__()
        
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.encoder_rnn_cell = nn.GRU(input_size, hidden_size)
        self.decoder_rnn_cell = nn.GRU(hidden_size*2, hidden_size)
        
        self.h2o = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=2)
        
        self.U = nn.Linear(self.hidden_size, self.hidden_size)
        self.W = nn.Linear(self.hidden_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size, 1)
        self.out2hidden = nn.Linear(self.output_size, self.hidden_size)   
        
        self.verbose = verbose
        
    def forward(self, input, max_output_chars = MAX_OUTPUT_CHARS, device = 'cpu', ground_truth = None):
        
        # encoder
        encoder_outputs, hidden = self.encoder_rnn_cell(input)
        encoder_outputs = encoder_outputs.view(-1, self.hidden_size)
        
        if self.verbose:
            print('Encoder output', encoder_outputs.shape)
        
        # decoder
        decoder_state = hidden
        decoder_input = torch.zeros(1, 1, self.output_size).to(device)
        
        outputs = []
        U = self.U(encoder_outputs)
        
        if self.verbose:
            print('Decoder state', decoder_state.shape)
            print('Decoder intermediate input', decoder_input.shape)
            print('U * Encoder output', U.shape)
        
        for i in range(max_output_chars):
            
            W = self.W(decoder_state.view(1, -1).repeat(encoder_outputs.shape[0], 1))
            V = self.attn(torch.tanh(U + W))
            attn_weights = F.softmax(V.view(1, -1), dim = 1) 
            
            if self.verbose:
                print('W * Decoder state', W.shape)
                print('V', V.shape)
                print('Attn', attn_weights.shape)
            
            attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))
            
            embedding = self.out2hidden(decoder_input)
            decoder_input = torch.cat((embedding[0], attn_applied[0]), 1).unsqueeze(0)
            
            if self.verbose:
                print('Attn LC', attn_applied.shape)
                print('Decoder input', decoder_input.shape)
                
            out, decoder_state = self.decoder_rnn_cell(decoder_input, decoder_state)
            
            if self.verbose:
                print('Decoder intermediate output', out.shape)
                
            out = self.h2o(decoder_state)
            out = self.softmax(out)
            outputs.append(out.view(1, -1))
            
            if self.verbose:
                print('Decoder output', out.shape)
                self.verbose = False
            
            max_idx = torch.argmax(out, 2, keepdim=True)
            if not ground_truth is None:
                max_idx = ground_truth[i].reshape(1, 1, 1)
            one_hot = torch.zeros(out.shape, device=device)
            one_hot.scatter_(2, max_idx, 1) 
            
            decoder_input = one_hot.detach()
            
        return outputs

In [28]:
#inference routine of sequence model.
def infer(net, eng_word,shape,device ='cpu'):
    # net.eval()
    input_ = word_rep(eng_word,eng_alpha2index,device) # convert the name into one hot encoding.
    outputs = net(input_,shape,device) # initilise the hidden layer.
    
    return outputs

In [51]:
net_attn = Transliteration_EncoderDecoder_Attention(len(eng_alpha2index), 256, len(hindi_alpha2index), verbose=True)

In [52]:
out = infer(net_attn, 'INDIA', 30)

Encoder output torch.Size([6, 256])
Decoder state torch.Size([1, 1, 256])
Decoder intermediate input torch.Size([1, 1, 129])
U * Encoder output torch.Size([6, 256])
W * Decoder state torch.Size([6, 256])
V torch.Size([6, 1])
Attn torch.Size([1, 6])
Attn LC torch.Size([1, 1, 256])
Decoder input torch.Size([1, 1, 512])
Decoder intermediate output torch.Size([1, 1, 256])
Decoder output torch.Size([1, 1, 129])


In [53]:
print(len(out))
for i in range(len(out)):
    print(out[i].shape, list(hindi_alpha2index.keys())[list(hindi_alpha2index.values()).index(torch.argmax(out[i]))])

30
torch.Size([1, 129]) भ
torch.Size([1, 129]) ॺ
torch.Size([1, 129]) ॺ
torch.Size([1, 129]) ॺ
torch.Size([1, 129]) ॺ
torch.Size([1, 129]) ॺ
torch.Size([1, 129]) ॺ
torch.Size([1, 129]) ॺ
torch.Size([1, 129]) ॺ
torch.Size([1, 129]) ॺ
torch.Size([1, 129]) ॺ
torch.Size([1, 129]) ॺ
torch.Size([1, 129]) ॺ
torch.Size([1, 129]) ॺ
torch.Size([1, 129]) ॺ
torch.Size([1, 129]) ॺ
torch.Size([1, 129]) ॺ
torch.Size([1, 129]) ॺ
torch.Size([1, 129]) ॺ
torch.Size([1, 129]) ॺ
torch.Size([1, 129]) ॺ
torch.Size([1, 129]) ॺ
torch.Size([1, 129]) ॺ
torch.Size([1, 129]) ॺ
torch.Size([1, 129]) ॺ
torch.Size([1, 129]) ॺ
torch.Size([1, 129]) ॺ
torch.Size([1, 129]) ॺ
torch.Size([1, 129]) ॺ
torch.Size([1, 129]) ॺ


## Training

In [32]:
def train_batch(net, opt, criterion, batch_size, device = 'cpu', teacher_force = False):
    
    net.train().to(device)
    opt.zero_grad()
    eng_batch, hindi_batch = train_data.get_batch(batch_size)
    
    total_loss = 0
    for i in range(batch_size):
        
        input = word_rep(eng_batch[i], eng_alpha2index, device)
        gt = gt_rep(hindi_batch[i], hindi_alpha2index, device)
        outputs = net(input, gt.shape[0], device, ground_truth = gt if teacher_force else None)
        
        for index, output in enumerate(outputs):
            loss = criterion(output, gt[index]) / batch_size
            loss.backward(retain_graph = True)
            total_loss += loss
        
    opt.step()
    return total_loss/batch_size

In [36]:
# Training Helper

def train_setup(net=net, config=None, momentum = 0.9, display_freq=5, device = 'cpu'):    #lr = 0.01, n_batches = 100, batch_size = 10,
    with wandb.init(config=config):
        config=wandb.config
        net = net.to(device)
        criterion = nn.NLLLoss(ignore_index = -1)
        opt = optim.Adam(net.parameters(), lr=.001)
        teacher_force_upto = config.n_batches//3

        loss_arr = np.zeros(config.n_batches + 1)

        for i in range(config.n_batches):
            loss_arr[i+1] = (loss_arr[i]*i + train_batch(net, opt, criterion, config.batch_size, device = device, teacher_force = i ))
            wandb.log({'loss':loss_arr[i+1], 'batch':i+1})
        
        return loss_arr

In [37]:
def train_setup1(net, lr = 0.001, n_batches = 100, batch_size = 10, momentum = 0.9, display_freq=5, device = 'cpu'):
    
    net = net.to(device)
    criterion = nn.NLLLoss(ignore_index = -1)
    opt = optim.Adam(net.parameters(), lr=lr)
    teacher_force_upto = n_batches//3
    
    loss_arr = np.zeros(n_batches + 1)
    
    for i in range(n_batches):
        loss_arr[i+1] = (loss_arr[i]*i + train_batch(net, opt, criterion, batch_size, device = device, teacher_force = i))
        
    return loss_arr

In [38]:
# Training without Attention

net = Transliteration_EncoderDecoder(len(eng_alpha2index), 256, len(hindi_alpha2index))

In [39]:
wandb.agent(sweep_id,train_setup,count=5)

wandb: Agent Starting Run: jap6qsvk with config:
wandb: 	batch_size: 64
wandb: 	cell_type: RNN
wandb: 	hidden_layer_size: 64
wandb: 	lr: 0.001
wandb: 	n_batches: 100


batch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,▁▁▁▁▁▁▁▁▁▁▁▁▁█
batch,100
loss,inf


wandb: Agent Starting Run: 4nyhleeu with config:
wandb: 	batch_size: 128
wandb: 	cell_type: GRU
wandb: 	hidden_layer_size: 64
wandb: 	lr: 0.001
wandb: 	n_batches: 10


batch,▁▂▃▃▄▅▆▆▇█
loss,▁▁▁▁▁▁▁▁▂█
batch,10
loss,197606.95312


wandb: Agent Starting Run: 81dyfmux with config:
wandb: 	batch_size: 64
wandb: 	cell_type: RNN
wandb: 	hidden_layer_size: 16
wandb: 	lr: 0.001
wandb: 	n_batches: 10


batch,▁▂▃▃▄▅▆▆▇█
loss,▁▁▁▁▁▁▁▁▂█
batch,10
loss,317166.96875


wandb: Agent Starting Run: inqh6erg with config:
wandb: 	batch_size: 128
wandb: 	cell_type: RNN
wandb: 	hidden_layer_size: 64
wandb: 	lr: 0.1
wandb: 	n_batches: 100


batch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,▁▁▁▁▁▁▁▁▁▁▁▁▁█
batch,100
loss,inf


wandb: Agent Starting Run: zqyxsxir with config:
wandb: 	batch_size: 128
wandb: 	cell_type: RNN
wandb: 	hidden_layer_size: 16
wandb: 	lr: 0.1
wandb: 	n_batches: 10


batch,▁▂▃▃▄▅▆▆▇█
loss,▁▁▁▁▁▁▁▁▂█
batch,10
loss,126312.71094


In [ ]:
!pip install wandb --upgrade

In [42]:
%%time
loss_history = train_setup1(net, lr=0.001, n_batches=100, batch_size = 64, display_freq=10, device = device)

Wall time: 7min 29s


In [43]:
loss_history

array([0.00000000e+00, 6.66395664e-01, 1.34522295e+00, 3.36074209e+00,
       1.07340269e+01, 4.35607147e+01, 2.18471237e+02, 1.31147717e+03,
       9.18096289e+03, 7.34483203e+04, 6.61035500e+05, 6.61035550e+06,
       7.27139120e+07, 8.72566912e+08, 1.13433702e+10, 1.58807179e+11,
       2.38210777e+12, 3.81137243e+13, 6.47933330e+14, 1.16628001e+16,
       2.21593203e+17, 4.43186419e+18, 9.30691469e+19, 2.04752118e+21,
       4.70929884e+22, 1.13023172e+24, 2.82557933e+25, 7.34650659e+26,
       1.98355684e+28, 5.55395935e+29, 1.61064824e+31, 4.83194458e+32,
       1.49790287e+34, 4.79328917e+35, 1.58178545e+37,            inf,
                  inf,            inf,            inf,            inf,
                  inf,            inf,            inf,            inf,
                  inf,            inf,            inf,            inf,
                  inf,            inf,            inf,            inf,
                  inf,            inf,            inf,            inf,
      

In [55]:
# Training with Attention

net_att = Transliteration_EncoderDecoder_Attention(len(eng_alpha2index), 256, len(hindi_alpha2index))

In [91]:
%%time
loss_history = train_setup1(net_att, lr=0.01, n_batches=100, batch_size = 64, display_freq=10, device = device)

Wall time: 18min 44s


In [92]:
loss_history

array([0.00000000e+00, 6.67277813e-01, 1.37211275e+00, 3.86458063e+00,
       1.26613417e+01, 5.25972786e+01, 2.64985687e+02, 1.59219141e+03,
       1.11481523e+04, 8.91884609e+04, 8.02699812e+05, 8.02700250e+06,
       8.82970320e+07, 1.05956442e+09, 1.37743370e+10, 1.92840712e+11,
       2.89261067e+12, 4.62817707e+13, 7.86790093e+14, 1.41622219e+16,
       2.69082209e+17, 5.38164432e+18, 1.13014534e+20, 2.48631970e+21,
       5.71853525e+22, 1.37244842e+24, 3.43112115e+25, 8.92091479e+26,
       2.40864704e+28, 6.74421180e+29, 1.95582147e+31, 5.86746440e+32,
       1.81891397e+34, 5.82052469e+35, 1.92077309e+37,            inf,
                  inf,            inf,            inf,            inf,
                  inf,            inf,            inf,            inf,
                  inf,            inf,            inf,            inf,
                  inf,            inf,            inf,            inf,
                  inf,            inf,            inf,            inf,
      

In [44]:
def test(net, word, device = 'cpu'):
    net = net.eval().to(device)
    outputs = infer(net, word, 30, device)
    hindi_output = ''
    for out in outputs:
        val, indices = out.topk(1)
        index = indices.tolist()[0][0]
        if index == 0:
            break
        hindi_char = hindi_alphabets[index+1]
        hindi_output += hindi_char
    print(word + ' - ' + hindi_output)
    return hindi_output

In [68]:
def calc_accuracy(net, device = 'cpu'):
    net = net.eval().to(device)
    predictions = []
    accuracy = 0
    for i in range(len(test_data)):
        eng, hindi = test_data[i]
        gt = gt_rep(hindi, hindi_alpha2index, device)
        outputs = infer(net, eng, gt.shape[0], device)
        correct = 0
        for index, out in enumerate(outputs):
            val, indices = out.topk(1)
            hindi_pos = indices.tolist()[0]
            print(hindi_pos[0],gt[index])
            if hindi_pos[0] == gt[index][0]:
                correct += 1
        
        accuracy += correct/gt.shape[0]
    accuracy /= len(test_data)
    return accuracy, outputs
     

In [69]:
accuracy,op = calc_accuracy(net) #* 100
# accuracy_attn = calc_accuracy(net_att) * 100
print('Accuracy w/o attention ', accuracy)
# print('Acurracy with attention', accuracy_attn)

57 tensor([38])
78 tensor([49])
49 tensor([47])
63 tensor([73])
0 tensor([22])
0 tensor([78])
0 tensor([57])
0 tensor([0])
57 tensor([57])
78 tensor([64])
49 tensor([23])
78 tensor([63])
49 tensor([16])
63 tensor([24])
0 tensor([63])
0 tensor([0])
57 tensor([51])
63 tensor([49])
49 tensor([78])
63 tensor([41])
0 tensor([0])
57 tensor([32])
78 tensor([78])
49 tensor([54])
78 tensor([64])
49 tensor([32])
63 tensor([49])
0 tensor([78])
0 tensor([57])
0 tensor([0])
57 tensor([37])
78 tensor([64])
49 tensor([49])
78 tensor([66])
49 tensor([41])
78 tensor([72])
49 tensor([51])
63 tensor([54])
0 tensor([72])
0 tensor([51])
0 tensor([65])
0 tensor([0])
57 tensor([8])
78 tensor([3])
49 tensor([34])
78 tensor([64])
49 tensor([43])
78 tensor([72])
49 tensor([3])
78 tensor([34])
0 tensor([72])
0 tensor([3])
0 tensor([57])
0 tensor([0])
57 tensor([57])
78 tensor([78])
49 tensor([43])
78 tensor([72])
49 tensor([55])
63 tensor([64])
0 tensor([48])
0 tensor([76])
0 tensor([3])
0 tensor([0])
57 tensor(

57 tensor([22])
78 tensor([76])
49 tensor([27])
78 tensor([72])
49 tensor([51])
63 tensor([63])
0 tensor([0])
57 tensor([51])
78 tensor([32])
49 tensor([22])
78 tensor([22])
49 tensor([49])
63 tensor([0])
57 tensor([43])
78 tensor([37])
49 tensor([78])
78 tensor([49])
49 tensor([63])
78 tensor([54])
49 tensor([51])
63 tensor([64])
0 tensor([48])
0 tensor([63])
0 tensor([3])
0 tensor([0])
57 tensor([43])
78 tensor([49])
49 tensor([64])
78 tensor([56])
49 tensor([39])
63 tensor([0])
57 tensor([57])
63 tensor([78])
49 tensor([43])
63 tensor([64])
0 tensor([41])
0 tensor([78])
0 tensor([29])
0 tensor([0])
57 tensor([6])
63 tensor([41])
49 tensor([55])
63 tensor([64])
0 tensor([39])
0 tensor([63])
0 tensor([0])
57 tensor([34])
78 tensor([64])
49 tensor([29])
63 tensor([65])
0 tensor([57])
0 tensor([57])
0 tensor([0])
57 tensor([57])
78 tensor([49])
49 tensor([63])
78 tensor([48])
49 tensor([47])
78 tensor([76])
49 tensor([58])
78 tensor([64])
49 tensor([10])
78 tensor([39])
0 tensor([78])
0

57 tensor([41])
63 tensor([24])
49 tensor([63])
63 tensor([34])
0 tensor([61])
0 tensor([63])
0 tensor([0])
57 tensor([10])
63 tensor([34])
49 tensor([61])
63 tensor([63])
0 tensor([41])
0 tensor([72])
0 tensor([3])
0 tensor([0])
57 tensor([22])
63 tensor([78])
49 tensor([51])
0 tensor([72])
0 tensor([41])
0 tensor([0])
57 tensor([29])
63 tensor([67])
49 tensor([51])
63 tensor([67])
0 tensor([57])
0 tensor([0])
57 tensor([22])
78 tensor([63])
49 tensor([49])
78 tensor([78])
49 tensor([48])
63 tensor([46])
0 tensor([63])
0 tensor([49])
0 tensor([0])
57 tensor([47])
63 tensor([3])
49 tensor([32])
63 tensor([76])
0 tensor([0])
57 tensor([43])
78 tensor([73])
49 tensor([47])
78 tensor([63])
49 tensor([8])
63 tensor([55])
0 tensor([0])
57 tensor([22])
63 tensor([76])
49 tensor([54])
63 tensor([64])
0 tensor([41])
0 tensor([63])
0 tensor([0])
57 tensor([43])
78 tensor([72])
49 tensor([3])
63 tensor([56])
49 tensor([41])
0 tensor([0])
57 tensor([18])
78 tensor([24])
49 tensor([57])
78 tensor(

57 tensor([22])
63 tensor([78])
49 tensor([49])
63 tensor([74])
0 tensor([57])
0 tensor([78])
0 tensor([45])
0 tensor([65])
0 tensor([0])
57 tensor([41])
78 tensor([63])
49 tensor([47])
78 tensor([65])
49 tensor([24])
78 tensor([64])
49 tensor([49])
78 tensor([63])
49 tensor([47])
0 tensor([65])
0 tensor([0])
57 tensor([58])
63 tensor([49])
49 tensor([78])
63 tensor([51])
0 tensor([72])
0 tensor([0])
57 tensor([29])
63 tensor([49])
63 tensor([78])
0 tensor([39])
0 tensor([63])
0 tensor([0])
57 tensor([51])
78 tensor([64])
49 tensor([16])
78 tensor([43])
49 tensor([78])
78 tensor([49])
49 tensor([40])
78 tensor([63])
49 tensor([41])
63 tensor([47])
0 tensor([3])
0 tensor([37])
0 tensor([78])
0 tensor([49])
0 tensor([65])
0 tensor([0])
57 tensor([46])
63 tensor([78])
49 tensor([49])
63 tensor([63])
0 tensor([3])
0 tensor([37])
0 tensor([64])
0 tensor([0])
57 tensor([17])
78 tensor([29])
49 tensor([72])
78 tensor([3])
49 tensor([57])
63 tensor([64])
0 tensor([48])
0 tensor([76])
0 tensor(

57 tensor([22])
78 tensor([63])
49 tensor([47])
78 tensor([63])
49 tensor([41])
63 tensor([63])
0 tensor([16])
0 tensor([3])
0 tensor([0])
57 tensor([27])
63 tensor([66])
49 tensor([32])
63 tensor([65])
0 tensor([51])
0 tensor([72])
0 tensor([0])
57 tensor([57])
78 tensor([3])
49 tensor([23])
78 tensor([78])
49 tensor([48])
63 tensor([63])
0 tensor([16])
0 tensor([3])
0 tensor([0])
57 tensor([22])
63 tensor([49])
49 tensor([3])
0 tensor([29])
0 tensor([0])
57 tensor([45])
78 tensor([49])
49 tensor([57])
63 tensor([63])
0 tensor([37])
0 tensor([63])
0 tensor([0])
57 tensor([43])
78 tensor([49])
49 tensor([64])
78 tensor([22])
49 tensor([51])
78 tensor([78])
49 tensor([43])
0 tensor([64])
0 tensor([37])
0 tensor([0])
57 tensor([58])
63 tensor([49])
49 tensor([43])
63 tensor([63])
0 tensor([51])
0 tensor([0])
57 tensor([47])
78 tensor([63])
49 tensor([11])
63 tensor([3])
0 tensor([32])
0 tensor([0])
57 tensor([24])
63 tensor([78])
49 tensor([49])
63 tensor([76])
0 tensor([54])
0 tensor([4

...

57 tensor([20])
63 tensor([34])
49 tensor([64])
63 tensor([57])
0 tensor([63])
0 tensor([0])
57 tensor([37])
63 tensor([76])
49 tensor([47])
63 tensor([49])
0 tensor([0])
57 tensor([49])
63 tensor([76])
49 tensor([43])
63 tensor([65])
0 tensor([0])
57 tensor([38])
78 tensor([43])
49 tensor([78])
78 tensor([43])
49 tensor([34])
78 tensor([61])
49 tensor([45])
78 tensor([63])
49 tensor([29])
0 tensor([61])
0 tensor([65])
0 tensor([0])
57 tensor([45])
63 tensor([63])
49 tensor([45])
63 tensor([37])
0 tensor([0])
57 tensor([6])
78 tensor([32])
49 tensor([78])
63 tensor([32])
49 tensor([58])
0 tensor([63])
0 tensor([57])
0 tensor([0])
57 tensor([22])
63 tensor([76])
49 tensor([51])
0 tensor([41])
0 tensor([0])
57 tensor([43])
78 tensor([78])
49 tensor([49])
78 tensor([37])
49 tensor([64])
78 tensor([51])
49 tensor([65])
63 tensor([32])
0 tensor([49])
0 tensor([0])
57 tensor([46])
63 tensor([67])
63 tensor([57])
0 tensor([65])
0 tensor([0])
57 tensor([58])
63 tensor([51])
49 tensor([64])
63 

0 tensor([0])
57 tensor([57])
78 tensor([78])
49 tensor([54])
78 tensor([49])
49 tensor([78])
78 tensor([24])
49 tensor([39])
0 tensor([67])
0 tensor([37])
0 tensor([0])
57 tensor([44])
63 tensor([72])
63 tensor([22])
0 tensor([63])
0 tensor([0])
57 tensor([8])
78 tensor([3])
49 tensor([34])
78 tensor([64])
49 tensor([43])
78 tensor([72])
49 tensor([41])
0 tensor([78])
0 tensor([34])
0 tensor([72])
0 tensor([32])
0 tensor([0])
57 tensor([32])
63 tensor([64])
49 tensor([3])
63 tensor([22])
0 tensor([63])
0 tensor([0])
57 tensor([24])
63 tensor([66])
49 tensor([49])
63 tensor([78])
0 tensor([39])
0 tensor([76])
0 tensor([3])
0 tensor([0])
57 tensor([29])
78 tensor([63])
49 tensor([37])
63 tensor([64])
63 tensor([48])
0 tensor([76])
0 tensor([3])
0 tensor([0])
57 tensor([57])
78 tensor([47])
49 tensor([41])
78 tensor([78])
49 tensor([54])
78 tensor([48])
49 tensor([22])
63 tensor([0])
57 tensor([57])
63 tensor([67])
63 tensor([45])
0 tensor([72])
0 tensor([0])
57 tensor([27])
78 tensor([3

57 tensor([22])
78 tensor([78])
49 tensor([56])
63 tensor([72])
49 tensor([37])
0 tensor([78])
0 tensor([49])
0 tensor([72])
0 tensor([3])
0 tensor([0])
57 tensor([16])
63 tensor([3])
49 tensor([22])
63 tensor([72])
0 tensor([41])
0 tensor([65])
0 tensor([0])
57 tensor([28])
63 tensor([72])
49 tensor([37])
63 tensor([78])
0 tensor([49])
0 tensor([0])
57 tensor([47])
78 tensor([66])
49 tensor([22])
63 tensor([49])
49 tensor([37])
0 tensor([72])
0 tensor([0])
57 tensor([17])
63 tensor([3])
49 tensor([33])
0 tensor([72])
0 tensor([0])
57 tensor([45])
63 tensor([63])
49 tensor([51])
0 tensor([78])
0 tensor([32])
0 tensor([65])
0 tensor([0])
57 tensor([23])
78 tensor([3])
49 tensor([24])
63 tensor([63])
49 tensor([51])
0 tensor([65])
0 tensor([0])
57 tensor([45])
78 tensor([65])
49 tensor([16])
78 tensor([57])
49 tensor([16])
63 tensor([57])
0 tensor([16])
0 tensor([57])
0 tensor([0])
57 tensor([45])
78 tensor([78])
49 tensor([3])
78 tensor([29])
49 tensor([29])
63 tensor([57])
0 tensor([47

57 tensor([39])
78 tensor([72])
49 tensor([54])
63 tensor([63])
63 tensor([43])
0 tensor([66])
0 tensor([49])
0 tensor([0])
57 tensor([47])
78 tensor([41])
49 tensor([54])
78 tensor([63])
49 tensor([48])
63 tensor([63])
0 tensor([0])
57 tensor([51])
78 tensor([32])
49 tensor([22])
63 tensor([22])
49 tensor([49])
0 tensor([0])
57 tensor([47])
78 tensor([72])
49 tensor([58])
63 tensor([47])
0 tensor([63])
0 tensor([41])
0 tensor([0])
57 tensor([41])
63 tensor([64])
63 tensor([48])
0 tensor([63])
0 tensor([0])
57 tensor([10])
78 tensor([40])
49 tensor([47])
63 tensor([43])
49 tensor([66])
0 tensor([49])
0 tensor([0])
57 tensor([6])
63 tensor([37])
49 tensor([58])
63 tensor([49])
0 tensor([0])
57 tensor([28])
78 tensor([66])
49 tensor([43])
63 tensor([37])
0 tensor([63])
0 tensor([0])
57 tensor([37])
78 tensor([66])
49 tensor([3])
78 tensor([24])
49 tensor([41])
63 tensor([63])
0 tensor([38])
0 tensor([0])
57 tensor([49])
78 tensor([25])
49 tensor([66])
78 tensor([43])
49 tensor([37])
63 t

57 tensor([6])
78 tensor([44])
49 tensor([57])
78 tensor([49])
49 tensor([63])
63 tensor([72])
0 tensor([3])
0 tensor([0])
57 tensor([57])
63 tensor([72])
63 tensor([58])
0 tensor([76])
0 tensor([0])
57 tensor([27])
63 tensor([45])
49 tensor([63])
63 tensor([37])
0 tensor([72])
0 tensor([0])
57 tensor([47])
63 tensor([64])
63 tensor([37])
0 tensor([64])
0 tensor([0])
57 tensor([45])
63 tensor([72])
49 tensor([51])
63 tensor([27])
0 tensor([63])
0 tensor([0])
57 tensor([57])
63 tensor([44])
49 tensor([61])
63 tensor([51])
0 tensor([37])
0 tensor([63])
0 tensor([0])
57 tensor([55])
63 tensor([45])
49 tensor([63])
63 tensor([41])
0 tensor([63])
0 tensor([0])
57 tensor([23])
78 tensor([63])
49 tensor([37])
78 tensor([67])
49 tensor([32])
78 tensor([76])
49 tensor([51])
63 tensor([63])
0 tensor([0])
57 tensor([55])
78 tensor([64])
49 tensor([54])
78 tensor([63])
49 tensor([57])
63 tensor([78])
0 tensor([37])
0 tensor([54])
0 tensor([0])
57 tensor([43])
78 tensor([78])
49 tensor([49])
78 ten

In [63]:
test_data1.head()

,0,1
0,thermax,थरमैक्स
1,sikhaaega,सिखाएगा
2,learn,लर्न
3,twitters,ट्विटर्स
4,tirunelveli,तिरुनेलवेली


In [49]:
test(net,'SOURABH')

SOURABH - ऺॏलीी


'ऺॏलीी'

In [56]:
accuracy = calc_accuracy(net_att) * 100
# accuracy_attn = calc_accuracy(net_att) * 100
print('Accuracy w/ attention ', accuracy)
# print('Acurracy with attention', accuracy_attn)

Accuracy w/ attention  1.9365594354237732


In [57]:
test(net_att,'SOURABH')

SOURABH - ुऺऺऺऺऺऺऺऺऺऺऺऺऺऺऺऺऺऺऺऺऺऺऺऺऺऺऺऺऺ


'ुऺऺऺऺऺऺऺऺऺऺऺऺऺऺऺऺऺऺऺऺऺऺऺऺऺऺऺऺऺ'